In [1]:
class PidController:

    def __init__(self, kp: float, ki: float, kd: float, out_min: float = -10000, out_max: float = 10000, window = 1):
        self.kp = kp
        self.ki = ki
        self.kd = kd
        self.out_min = out_min
        self.out_max = out_max
        self.window = window
        self.__prev_error = 0
        self.__last_index = 0
        self.__p = 0
        self.__i = 0
        self.__d = 0
        self.__err_history = window * [0]

    @property
    def p(self):
        return self.__p

    @property
    def i(self):
        return self.__i

    @property
    def d(self):
        return self.__d

    def get_output(self, set_point: float, process_value: float, dt: float) -> float:
        self.__err_history[self.__last_index] = set_point - process_value
        self.__last_index = (self.__last_index + 1) % self.window
        error = sum(self.__err_history) / self.window
        
        self.__p = self.kp *  error
        self.__i += self.ki * error * dt
        self.__d = self.kd * (self.__prev_error - error) / dt
        self.__prev_error = error

        output = self.p + self.i + self.d
        if self.out_min > output or output > self.out_max:
            self.__i -= self.ki * error * dt
        output = max(self.out_min, min(self.out_max, output))
        return output
    
    def step_response(self, dt: float, duration=1):
        points = round(duration / dt)
        self.__i = 0
        set_point = 1
        history = []
        history.append(0)
        for i in range(points):
            process_value = self.get_output(set_point, process_value, dt)
            history.append(process_value + history[-1])
        return history



